<img src="pr1.png" />

In [10]:
import cvxpy
import gurobipy
from cvxpy import *
import numpy as np

In [18]:
np.random.seed(42)

# initial matrix
m = np.array([[ 5 , 5 , 10, 10, 10 ],
              [ 5 , 5 , 10, 20, 10 ],
              [ 0 , 5 , 5 , 10,  5 ],
              [ 0 , 0 , 0 , 5 ,  0 ]])

N, M = m.shape
C = N * M

# delta matrix
d = np.ones((N, M)) * 6
d = d - m

# cost to transfer the ground from the (i1, j1) to (i2, j2)
def cost(i1, j1, i2, j2):
    assert(i1 < N and i2 < N and j1 < M and j2 < M and 
           i1 >= 0 and i2 >= 0 and j1 >= 0 and j2 >= 0)
    
    if i1 == i2 and j1 == j2:
        return 1000000
    
    return np.sqrt(np.power(np.abs(i1 - i2), 2) + np.power(np.abs(j1 - j2), 2))

m = m.reshape(C,1)
d = d.reshape(C,1)

f = Int(C * C)
c = np.zeros(C * C)

# fill the cost matrix
for i1 in xrange(N):
    for j1 in xrange(M):
        for i2 in xrange(N):
            for j2 in xrange(M):
                c[(i1 * M + j1) * C + (i2 * M + j2)] = cost(i1, j1, i2, j2)
                
# Each cell can either be only a consumer or a "donor" in my model.
# This simplification just introduces a new cutting plane which doesn't
# affect the global optimum (I had the same result without "donors").
# This restriction gave me more integers when I was trying to solve with 
# the general LP solver.
def pred(x):
    if x < 0:
        return 1
    
    return 0

pd = np.vectorize(pred)
is_donor = pd(d).reshape(C,)

flow_network_constraints = [ (-is_donor[i] * cvxpy.sum_entries(f[i*C:i*C + C]) + \
                             (1 - is_donor[i]) * cvxpy.sum_entries(f[i:C*C:C]) == d[i]) \
                            for i in xrange(C) ]
fnc_not_donors = [ (1 - is_donor[i]) * cvxpy.sum_entries(f[i*C:i*C + C]) == 0 for i in xrange(C) ]
fnc_donors = [ is_donor[i] * cvxpy.sum_entries(f[i:C*C:C]) == 0 for i in xrange(C) ]

flows_are_positive = [ (f[i] >= 0) for i in xrange(C * C) ]

obj = Minimize(c * f)

constraints = flows_are_positive + flow_network_constraints + fnc_not_donors + fnc_donors

#print installed_solvers()

prob = Problem(obj, constraints)
res = prob.solve(feastol=1e-8) # verbose=True, 
print "Result: ", res
eps = 0.0000000001
print "Flows size: ", f.value[np.where( f.value > eps )] 
# print cvxpy.sum_entries(f[13:C*C:C]).value

Result:  95.4910638371
Flows size:  [[ 1.  1.  1.  1.  4.  1.  3.  2.  1.  1.  1.  5.  6.  2.  1.  3.  4.]]


I've tried several similar min cost max flow models. All of them gave the same result as this one that is constrained to have an integer valued flows.

<img src="pr2.png" />

Each of our clients has a constraint $\sum_j{cf_j} = 1$, where $cf$ is the client/facility arrangement matrix. If there is more than one non-zero $cf_j$ for one client we will take the greatest one and try to branch on it. If each of the clients belongs to exactly one facility than we should also check that the opened/closed facility vector $f$  also got no fractional values. In case it has we will make a branch on it too.

In [87]:
# initialization

import numpy as np
import scipy as sp
from numpy import random
from numpy import matlib
import matplotlib.pyplot as plt
from cvxpy import *
import cvxpy

%matplotlib inline

np.random.seed(1)

C = 40; # number of clients
clients = np.random.rand(2,C); #client positions
F = 20; #number of facilities
facilities = np.random.rand(2,F);

capacities = np.array([2, 3, 4, 2, 5, 3, 2, 1, 2, 3, 4, 3, 2, 1, 2, 2, 2, 3, 2, 3]);

dx = np.matlib.repmat(clients[0,:],F,1) - np.matlib.repmat(facilities[0,:],C,1).transpose();
dy = np.matlib.repmat(clients[1,:],F,1) - np.matlib.repmat(facilities[1,:],C,1).transpose();

assignment_costs = 3*(dx*dx + dy*dy)

opening_costs = np.array([1, 2, 1, 2, 1, 1.2, 1.3, 2, 1, 1, 1.4, 2, 1, 1, 1, 1, 1.5, 1.3, 2, 1]);

In [129]:
POS_INF = 1000000L
eps = 1e-8

"""
Solves the problem for the specific node.
Node is identified by some fixed client/facility mapping integer constraints.
And also with opened/closed facilities constraints.

Ex: constr = [({1: (2, 0), 2: (1, 1)}, {4: 0, 5: 1})] - 
    * 1st client doesn't belong to 2nd facility
    * 2nd client belongs to 1st facility
    * 4th facility is closed
    * 5th is opened
"""
def lp_solve(constr):
    global F, C, nodes_visited, clients, facilities, capacities, assignment_costs, opening_costs
    
    # closed facilities
    closed = map(lambda (a,b): a, filter(lambda (a,b): b == 0, constr[1].iteritems()))

    # check if there's enough total capacity of the facilities
    if np.sum(capacities) - np.sum(capacities[closed]) < C: 
        return [POS_INF, []]
    
    f = Variable(F)
    cf = Int(C, F)
            
    constraints = [ f[i] >= 0 for i in xrange(F) ] + \
                  [ f[i] <= 1 for i in xrange(F) ] + \
                  [ (cvxpy.sum_entries(cf[i,:]) == 1) for i in xrange(C) ] + \
                  [ (cvxpy.sum_entries(cf[:,i]) == f[i] * capacities[i]) for i in xrange(F) ] + \
                  [ cf[i, j] >= 0 for i in xrange(C) for j in xrange(F) ]

    additional_constr = []
    
    for k, v in constr[0].iteritems():
        additional_constr += [ cf[k, v[0]] == v[1] ]
    
    for k, v in constr[1].iteritems():
        additional_constr += [ f[k] == v ]
        
    obj = Minimize(vec(f).T * opening_costs + vec(cf).T * vec(assignment_costs))
    prob = Problem(obj, constraints + additional_constr)
    res = prob.solve()
    eps = 1e-10
    #opened = np.where(f.value > eps)[0]
    
    return [res, cf.value, f.value]

def select_f_frac(f):
    assert(len(f) == F)
    
    mx = -1
    mi = -1
    
    for i in xrange(F):
        if (f[i] <= 1 - eps and f[i] >= eps):
            if mx < f[i]:
                mx = f[i]
                mi = i
    
    return mi
    
def select_frac(x):
    assert(x.shape == (C, F))
    res = 0
    mx = -1
    mi, mj = -1, -1
    
    for i in xrange(C):
        for j in xrange(F):
            if (x[i, j] <= 1 - eps and x[i, j] >= eps):
                if mx < x[i, j]:
                    mx = x[i, j]
                    mi = i
                    mj = j
                
    return (mi, mj)

def branch_and_bound(iterations = 1):
    incumbent_value = POS_INF
    incumbent = 0
    
    queue = [({}, {})]
    
    while queue and iterations > 0:
        iterations -= 1
        node = queue.pop(0)
        res, x, f_val = lp_solve(node)

        if res >= incumbent_value:
            continue
        
        idx = select_frac(x)
        
        if idx == (-1, -1):
            idx = select_f_frac(f_val)
            
            if idx == -1:
                print "UPDATED: ", res
                incumbent_value = res
                incumbent = (x, f_val)
            else:
                nc_0, nc_1 = (node[0].copy(), node[1].copy()), (node[0].copy(), node[1].copy())
                nc_0[1][idx] = 0
                nc_1[1][idx] = 1
                queue.extend([nc_0, nc_1])
                
        else:
            nc_1, nc_0 = (node[0].copy(), node[1].copy()), (node[0].copy(), node[1].copy())
            nc_1[0][idx[0]] = (idx[1], 1)
            nc_0[0][idx[0]] = (idx[1], 0)
            queue.extend([nc_0, nc_1])
        
        print "Node: ", node, "; res: ", res
    print "======================================="
    print "Best result: "
    print incumbent_value
    print "Facilities: ", incumbent[1]
    np.set_printoptions(precision = 1, suppress = True) 
    print "Arrangement: ", incumbent[0]
        
branch_and_bound(100)

Node:  ({}, {}) ; res:  21.9517965463
Node:  ({}, {1: 0}) ; res:  21.9787751434
Node:  ({}, {1: 1}) ; res:  22.1517076039
Node:  ({}, {1: 0, 11: 0}) ; res:  21.9950091881
UPDATED:  22.1533386498
Node:  ({}, {1: 0, 11: 1}) ; res:  22.1533386498
Best result: 
22.1533386498
Facilities:  [[  1.00000000e+00]
 [  1.26035731e-14]
 [  1.00000000e+00]
 [  5.93423416e-12]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  4.38803800e-12]
 [  5.27314475e-12]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.54998494e-11]
 [  1.00000000e+00]
 [  4.14184956e-12]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]]
Arrangement:  [[-0. -0.  0.  0.  0.  0.  0. -0.  0. -0.  1.  0.  0.  0. -0. -0.  0.  0.
  -0. -0.]
 [-0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0. -0.  1.
  -0. -0.]
 [ 0. -0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.
   1. -0.]
 [-0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.

Previous solution via simulated annealing gave me $22.89$ last time. Now it's $22.15$. And GUROBI gives $\approx 21.8$

<img src="pr3.png" />